<a href="https://colab.research.google.com/github/lkhok22/ML-FinalProject-Walmart-Recruiting---Store-Sales-Forecasting/blob/main/model_experiment_ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import libraries and Initialize wandb

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import zipfile
import glob
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')

# Initialize wandb
import wandb
wandb.init(project="walmart-sales-forecasting", name="arima-model")

# Mount Google Drive
drive.mount('/content/drive')

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lkhok22 (lkhok22-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Mounted at /content/drive


#Unzip and load data

In [ ]:
# Unzip dataset
zip_path = "/content/drive/MyDrive/ML-FinalProject/data.zip"
extract_path = "/content/data"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
for zip_file in glob.glob(f"{extract_path}/*.csv.zip"):
    with zipfile.ZipFile(zip_file, 'r') as z:
        z.extractall(extract_path)

# Load data
train = pd.read_csv(f"{extract_path}/train.csv")
test = pd.read_csv(f"{extract_path}/test.csv")
stores = pd.read_csv(f"{extract_path}/stores.csv")
features = pd.read_csv(f"{extract_path}/features.csv")

# Log data loading completion to wandb
wandb.log({"step": "data_loading", "status": "completed"})

#Process data

In [2]:
# Convert Date column to datetime
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])
features['Date'] = pd.to_datetime(features['Date'])

# Merge train with features and stores
train_merged = train.merge(features, on=['Store', 'Date', 'IsHoliday'], how='left')
train_merged = train_merged.merge(stores, on='Store', how='left')

# Merge test with features and stores
test_merged = test.merge(features, on=['Store', 'Date', 'IsHoliday'], how='left')
test_merged = test_merged.merge(stores, on='Store', how='left')

# Sort by Store, Dept, and Date
train_merged = train_merged.sort_values(['Store', 'Dept', 'Date'])
test_merged = test_merged.sort_values(['Store', 'Dept', 'Date'])

# Log preprocessing step to wandb
wandb.log({"step": "data_preprocessing", "status": "merged_and_sorted"})

# Display basic info to verify
print("Train merged shape:", train_merged.shape)
print("Test merged shape:", test_merged.shape)
print(train_merged.head())

Train merged shape: (421570, 16)
Test merged shape: (115064, 15)
   Store  Dept       Date  Weekly_Sales  IsHoliday  Temperature  Fuel_Price  \
0      1     1 2010-02-05      24924.50      False        42.31       2.572   
1      1     1 2010-02-12      46039.49       True        38.51       2.548   
2      1     1 2010-02-19      41595.55      False        39.93       2.514   
3      1     1 2010-02-26      19403.54      False        46.63       2.561   
4      1     1 2010-03-05      21827.90      False        46.50       2.625   

   MarkDown1  MarkDown2  MarkDown3  MarkDown4  MarkDown5         CPI  \
0        NaN        NaN        NaN        NaN        NaN  211.096358   
1        NaN        NaN        NaN        NaN        NaN  211.242170   
2        NaN        NaN        NaN        NaN        NaN  211.289143   
3        NaN        NaN        NaN        NaN        NaN  211.319643   
4        NaN        NaN        NaN        NaN        NaN  211.350143   

   Unemployment Type    Siz

In [3]:
# Group train data by Store and Dept to check number of observations
train_grouped = train_merged.groupby(['Store', 'Dept']).size().reset_index(name='count')

# Filter Store-Dept pairs with at least 10 observations
valid_pairs = train_grouped[train_grouped['count'] >= 10][['Store', 'Dept']]
print(f"Number of valid Store-Dept pairs: {len(valid_pairs)}")

# Filter train_merged to include only valid Store-Dept pairs
train_filtered = train_merged.merge(valid_pairs, on=['Store', 'Dept'], how='inner')

# Handle missing Weekly_Sales in train_filtered (if any)
# Use last-value strategy: forward fill within each Store-Dept group
train_filtered['Weekly_Sales'] = train_filtered.groupby(['Store', 'Dept'])['Weekly_Sales'].ffill()

# Check for any remaining missing Weekly_Sales
missing_sales = train_filtered['Weekly_Sales'].isna().sum()
print(f"Missing Weekly_Sales after ffill: {missing_sales}")

# If there are still missing Weekly_Sales (e.g., at the start of a series), fill with group mean
if missing_sales > 0:
    mean_sales = train_filtered.groupby(['Store', 'Dept'])['Weekly_Sales'].transform('mean')
    train_filtered['Weekly_Sales'] = train_filtered['Weekly_Sales'].fillna(mean_sales)

# Log missing data handling to wandb
wandb.log({"step": "missing_data_handling", "missing_sales_after_ffill": missing_sales})

# Verify the filtered dataset
print("Train filtered shape:", train_filtered.shape)
print(train_filtered.head())

Number of valid Store-Dept pairs: 3167
Missing Weekly_Sales after ffill: 0
Train filtered shape: (420927, 16)
   Store  Dept       Date  Weekly_Sales  IsHoliday  Temperature  Fuel_Price  \
0      1     1 2010-02-05      24924.50      False        42.31       2.572   
1      1     1 2010-02-12      46039.49       True        38.51       2.548   
2      1     1 2010-02-19      41595.55      False        39.93       2.514   
3      1     1 2010-02-26      19403.54      False        46.63       2.561   
4      1     1 2010-03-05      21827.90      False        46.50       2.625   

   MarkDown1  MarkDown2  MarkDown3  MarkDown4  MarkDown5         CPI  \
0        NaN        NaN        NaN        NaN        NaN  211.096358   
1        NaN        NaN        NaN        NaN        NaN  211.242170   
2        NaN        NaN        NaN        NaN        NaN  211.289143   
3        NaN        NaN        NaN        NaN        NaN  211.319643   
4        NaN        NaN        NaN        NaN        Na

In [4]:
# Import required libraries for stationarity test and ARIMA
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm

# Function to check stationarity using ADF test
def check_stationarity(series, store, dept):
    result = adfuller(series.dropna(), autolag='AIC')
    p_value = result[1]
    is_stationary = p_value < 0.05  # 5% significance level
    wandb.log({
        f"stationarity_store_{store}_dept_{dept}": {
            "p_value": p_value,
            "is_stationary": is_stationary
        }
    })
    return is_stationary

# Create dictionary to store time series for each Store-Dept pair
time_series_dict = {}

# Iterate over valid Store-Dept pairs
for _, row in valid_pairs.iterrows():
    store, dept = row['Store'], row['Dept']
    # Extract time series for this Store-Dept pair
    ts = train_filtered[(train_filtered['Store'] == store) &
                       (train_filtered['Dept'] == dept)][['Date', 'Weekly_Sales']]
    # Set Date as index
    ts = ts.set_index('Date')['Weekly_Sales']
    # Check for sufficient data
    if len(ts) >= 10:
        # Check stationarity
        is_stationary = check_stationarity(ts, store, dept)
        time_series_dict[(store, dept)] = {
            'series': ts,
            'is_stationary': is_stationary
        }

# Count stationary and non-stationary series
stationary_count = sum(1 for v in time_series_dict.values() if v['is_stationary'])
print(f"Number of stationary series: {stationary_count}")
print(f"Number of non-stationary series: {len(time_series_dict) - stationary_count}")

# Log stationarity check to wandb
wandb.log({
    "step": "stationarity_check",
    "stationary_series_count": stationary_count,
    "total_series_count": len(time_series_dict)
})

Number of stationary series: 2275
Number of non-stationary series: 892


In [5]:
# Import ARIMA and evaluation metrics
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')  # Suppress statsmodels warnings

# Function to calculate WMAE
def calculate_wmae(y_true, y_pred, is_holiday):
    weights = np.where(is_holiday, 5, 1)
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

# Define train-validation split and train ARIMA on a subset
val_predictions = []
arima_models = {}
arima_order = (1, 0, 2)

# Limit to a subset of Store-Dept pairs for faster testing (e.g., first 10 pairs)
subset_pairs = list(time_series_dict.keys())[:10]
print(f"Training ARIMA on {len(subset_pairs)} Store-Dept pairs")

# Iterate over subset of Store-Dept pairs
for store, dept in subset_pairs:
    ts = time_series_dict[(store, dept)]['series']
    # Set frequency to W-FRI to avoid warnings
    ts.index = ts.index.to_period('W-FRI').to_timestamp('W-FRI')

    # Sort dates and split into train (first 80%) and validation (last 20%)
    dates = ts.index.sort_values()
    train_size = int(0.8 * len(dates))
    train_dates = dates[:train_size]
    val_dates = dates[train_size:]

    train_ts = ts[train_dates]
    val_ts = ts[val_dates]

    print(f"Store {store}, Dept {dept}: Train size = {len(train_ts)}, Val size = {len(val_ts)}")

    if len(train_ts) >= 10 and len(val_ts) > 0:
        try:
            # Train ARIMA model
            model = ARIMA(train_ts, order=arima_order).fit()
            arima_models[(store, dept)] = model

            # Predict on validation set
            val_pred = model.forecast(steps=len(val_ts))

            # Get actual values and holiday flags for validation
            val_actual = val_ts.values
            val_holidays = train_filtered[(train_filtered['Store'] == store) &
                                        (train_filtered['Dept'] == dept) &
                                        (train_filtered['Date'].isin(val_dates))]['IsHoliday'].values

            # Ensure holiday flags match prediction length
            if len(val_holidays) != len(val_pred):
                print(f"Warning: Holiday flags length mismatch for Store {store}, Dept {dept}")
                continue

            # Calculate WMAE for this Store-Dept pair
            wmae = calculate_wmae(val_actual, val_pred, val_holidays)

            # Store predictions for overall evaluation
            for date, pred, actual, holiday in zip(val_ts.index, val_pred, val_actual, val_holidays):
                val_predictions.append({
                    'Store': store,
                    'Dept': dept,
                    'Date': date,
                    'Weekly_Sales_Pred': pred,
                    'Weekly_Sales_Actual': actual,
                    'IsHoliday': holiday
                })

            # Log WMAE for this pair
            wandb.log({f'wmae_store_{store}_dept_{dept}': wmae})

        except Exception as e:
            print(f"ARIMA failed for Store {store}, Dept {dept}: {e}")
            # Use last-value strategy for failed models
            last_value = train_ts[-1] if len(train_ts) > 0 else train_filtered[train_filtered['Dept'] == dept]['Weekly_Sales'].mean()
            val_pred = np.full(len(val_ts), last_value)
            val_holidays = train_filtered[(train_filtered['Store'] == store) &
                                        (train_filtered['Dept'] == dept) &
                                        (train_filtered['Date'].isin(val_dates))]['IsHoliday'].values
            if len(val_holidays) != len(val_pred):
                print(f"Warning: Holiday flags length mismatch in fallback for Store {store}, Dept {dept}")
                continue
            for date, pred, actual, holiday in zip(val_ts.index, val_pred, val_actual, val_holidays):
                val_predictions.append({
                    'Store': store,
                    'Dept': dept,
                    'Date': date,
                    'Weekly_Sales_Pred': pred,
                    'Weekly_Sales_Actual': actual,
                    'IsHoliday': holiday
                })

# Convert predictions to DataFrame
val_predictions_df = pd.DataFrame(val_predictions)

# Check if predictions DataFrame is empty
if val_predictions_df.empty:
    print("Error: val_predictions_df is empty. Check if any Store-Dept pairs had valid validation data.")
else:
    print("val_predictions_df columns:", val_predictions_df.columns.tolist())
    print(val_predictions_df.head())

# Calculate overall WMAE if DataFrame is not empty
if not val_predictions_df.empty:
    overall_wmae = calculate_wmae(val_predictions_df['Weekly_Sales_Actual'],
                                 val_predictions_df['Weekly_Sales_Pred'],
                                 val_predictions_df['IsHoliday'])
    print(f"Overall WMAE for ARIMA(1,0,2) on subset: {overall_wmae}")

    # Log overall WMAE to wandb
    wandb.log({"step": "arima_training_subset", "arima_order": "1,0,2", "overall_wmae": overall_wmae})
else:
    print("Skipping WMAE calculation due to empty predictions DataFrame.")

Training ARIMA on 10 Store-Dept pairs
Store 1, Dept 1: Train size = 114, Val size = 29
Store 1, Dept 2: Train size = 114, Val size = 29
Store 1, Dept 3: Train size = 114, Val size = 29
Store 1, Dept 4: Train size = 114, Val size = 29
Store 1, Dept 5: Train size = 114, Val size = 29
Store 1, Dept 6: Train size = 114, Val size = 29
Store 1, Dept 7: Train size = 114, Val size = 29
Store 1, Dept 8: Train size = 114, Val size = 29
Store 1, Dept 9: Train size = 114, Val size = 29
Store 1, Dept 10: Train size = 114, Val size = 29
val_predictions_df columns: ['Store', 'Dept', 'Date', 'Weekly_Sales_Pred', 'Weekly_Sales_Actual', 'IsHoliday']
   Store  Dept       Date  Weekly_Sales_Pred  Weekly_Sales_Actual  IsHoliday
0      1     1 2012-04-13       47378.974270             34684.21      False
1      1     1 2012-04-20       29380.548784             16976.19      False
2      1     1 2012-04-27       24169.315220             16347.60      False
3      1     1 2012-05-04       23549.203121        

In [6]:
# Prepare test set predictions
test_predictions = []

# Iterate over test set Store-Dept-Date triplets
test_grouped = test_merged.groupby(['Store', 'Dept'])
for (store, dept), group in test_grouped:
    # Get test dates for this Store-Dept pair
    test_dates = group['Date'].sort_values()
    test_holidays = group['IsHoliday'].values

    # Check if we have a trained model for this Store-Dept pair
    if (store, dept) in arima_models:
        try:
            model = arima_models[(store, dept)]
            # Forecast for the number of test dates
            pred = model.forecast(steps=len(test_dates))
        except Exception as e:
            print(f"Forecast failed for Store {store}, Dept {dept}: {e}")
            # Fallback to last-value or mean
            train_ts = time_series_dict.get((store, dept), {}).get('series', None)
            pred = np.full(len(test_dates), train_ts[-1] if train_ts is not None and len(train_ts) > 0 else
                          train_filtered[train_filtered['Dept'] == dept]['Weekly_Sales'].mean())
    else:
        # For new Store-Dept pairs in test set, use department mean
        train_ts = time_series_dict.get((store, dept), {}).get('series', None)
        pred = np.full(len(test_dates), train_ts[-1] if train_ts is not None and len(train_ts) > 0 else
                      train_filtered[train_filtered['Dept'] == dept]['Weekly_Sales'].mean())

    # Store predictions
    for date, pred_sales in zip(test_dates, pred):
        test_predictions.append({
            'Store': store,
            'Dept': dept,
            'Date': date,
            'Weekly_Sales': pred_sales
        })

# Convert to DataFrame
test_predictions_df = pd.DataFrame(test_predictions)

# Create Id column in the format Store_Dept_Date
test_predictions_df['Id'] = test_predictions_df.apply(
    lambda x: f"{int(x['Store'])}_{int(x['Dept'])}_{x['Date'].strftime('%Y-%m-%d')}", axis=1)

# Prepare submission file
submission = test_predictions_df[['Id', 'Weekly_Sales']]
submission['Weekly_Sales'] = submission['Weekly_Sales'].round(2)
submission.to_csv('/content/submission.csv', index=False)

# Log submission file creation to wandb
wandb.log({"step": "test_predictions", "submission_file": "created"})
wandb.save('/content/submission.csv')

# Verify submission
print("Submission file shape:", submission.shape)
print(submission.head())

wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


Submission file shape: (115064, 2)
               Id  Weekly_Sales
0  1_1_2012-11-02      47378.97
1  1_1_2012-11-09      29380.55
2  1_1_2012-11-16      24169.32
3  1_1_2012-11-23      23549.20
4  1_1_2012-11-30      23475.41


In [7]:
# Import required libraries
import numpy as np
import pandas as pd
import wandb

# Generate test set predictions with robust fallback
test_predictions = []

# Global mean as ultimate fallback if department mean is NaN
global_mean_sales = train_filtered['Weekly_Sales'].mean() if not train_filtered['Weekly_Sales'].isna().all() else 0.0
print(f"Global mean sales (fallback): {global_mean_sales}")

# Check departments in test set not in train set
train_depts = set(train_filtered['Dept'].unique())
test_depts = set(test_merged['Dept'].unique())
missing_depts = test_depts - train_depts
print(f"Departments in test set but not in train set: {missing_depts}")

# Iterate over test set Store-Dept-Date triplets
test_grouped = test_merged.groupby(['Store', 'Dept'])
for (store, dept), group in test_grouped:
    test_dates = group['Date'].sort_values()
    test_holidays = group['IsHoliday'].values

    # Initialize predictions
    pred = None

    # Check if we have a trained model for this Store-Dept pair
    if (store, dept) in arima_models:
        try:
            model = arima_models[(store, dept)]
            pred = model.forecast(steps=len(test_dates))
        except Exception as e:
            print(f"Forecast failed for Store {store}, Dept {dept}: {e}")

    # If no model or forecast failed, use fallback
    if pred is None or np.any(np.isnan(pred)) or np.any(np.isinf(pred)):
        train_ts = time_series_dict.get((store, dept), {}).get('series', None)
        if train_ts is not None and len(train_ts) > 0:
            pred = np.full(len(test_dates), train_ts[-1])
        else:
            # Use department mean if available, otherwise global mean
            dept_mean = train_filtered[train_filtered['Dept'] == dept]['Weekly_Sales'].mean()
            pred = np.full(len(test_dates), dept_mean if not np.isnan(dept_mean) else global_mean_sales)

    # Ensure predictions are valid floats and round to two decimal places
    pred = np.where(np.isnan(pred) | np.isinf(pred), global_mean_sales, pred)
    pred = np.round(pred, 2).astype(float)  # Round to nearest hundredth and ensure float

    # Store predictions
    for date, pred_sales in zip(test_dates, pred):
        test_predictions.append({
            'Store': store,
            'Dept': dept,
            'Date': date,
            'Weekly_Sales': pred_sales
        })

# Convert to DataFrame
test_predictions_df = pd.DataFrame(test_predictions)

# Create Id column in the format Store_Dept_Date
test_predictions_df['Id'] = test_predictions_df.apply(
    lambda x: f"{int(x['Store'])}_{int(x['Dept'])}_{x['Date'].strftime('%Y-%m-%d')}", axis=1)

# Prepare submission file with rounded Weekly_Sales
submission = test_predictions_df[['Id', 'Weekly_Sales']]
submission['Weekly_Sales'] = submission['Weekly_Sales'].round(2)  # Ensure rounding in submission
submission['Weekly_Sales'] = submission['Weekly_Sales'].fillna(global_mean_sales)  # Final NaN check
submission['Weekly_Sales'] = submission['Weekly_Sales'].apply(lambda x: global_mean_sales if pd.isna(x) or np.isinf(x) else x)

# Verify no empty strings or invalid values
submission['Weekly_Sales'] = submission['Weekly_Sales'].apply(lambda x: global_mean_sales if isinstance(x, str) and x.strip() == '' else x)

# Save submission file
submission.to_csv('/content/submission.csv', index=False)

# Log submission file creation to wandb
wandb.log({"step": "test_predictions_fixed_rounded", "submission_file": "created"})
wandb.save('/content/submission.csv')

# Verify submission
print("Submission file shape:", submission.shape)
print(submission.head())
print("\nNaN values in Weekly_Sales:", submission['Weekly_Sales'].isna().sum())
print("Infinite values in Weekly_Sales:", np.isinf(submission['Weekly_Sales']).sum())
print("Empty or non-numeric values in Weekly_Sales:",
      submission['Weekly_Sales'].apply(lambda x: isinstance(x, str) and x.strip() == '').sum())

# Check problematic rows in new submission
problem_lines = [4925, 9497, 36405, 47110, 49709, 49710, 49711, 49712, 49713, 49714]
problem_rows = [line - 2 for line in problem_lines]  # Adjust for header and 0-based indexing
print("\nChecking problematic rows in new submission:")
print(submission.iloc[problem_rows])

Global mean sales (fallback): 16005.536820398784
Departments in test set but not in train set: {np.int64(43), np.int64(77), np.int64(39)}
Submission file shape: (115064, 2)
               Id  Weekly_Sales
0  1_1_2012-11-02      47378.97
1  1_1_2012-11-09      29380.55
2  1_1_2012-11-16      24169.32
3  1_1_2012-11-23      23549.20
4  1_1_2012-11-30      23475.41

NaN values in Weekly_Sales: 0
Infinite values in Weekly_Sales: 0
Empty or non-numeric values in Weekly_Sales: 0

Checking problematic rows in new submission:
                     Id  Weekly_Sales
4923    2_77_2013-01-11      16005.54
9495    4_39_2013-07-12      16005.54
36403  14_43_2012-12-07      16005.54
47108  18_43_2013-04-26      16005.54
49707  19_39_2012-11-02      16005.54
49708  19_39_2012-11-09      16005.54
49709  19_39_2012-11-16      16005.54
49710  19_39_2012-11-23      16005.54
49711  19_39_2012-11-30      16005.54
49712  19_39_2012-12-07      16005.54


#Score: 4901.43392